In [3]:
import pandas as pd #principal herramienta de gestion de datasets como dataframe
import numpy as np #para trabajar con arrays y algunas cuestiones estadisticas
from sklearn import preprocessing #procesamiento de detasets 
from sklearn.metrics import mean_squared_error #herramienta estadistica para medir errores
import seaborn as sns #herramienta grafica
sns.set()
from sklearn.model_selection import train_test_split #para separar el dataset en entrenamiento y testeo
from sklearn.tree import DecisionTreeClassifier #modelo de arbol
from sklearn.metrics import confusion_matrix #para crear la matriz de confusión
from sklearn.metrics import ConfusionMatrixDisplay #para graficar la matriz de confusión
from sklearn.model_selection import GridSearchCV #para optimización de hiperparámetro
import joblib #airve para bajar el modelo entrenado y guardarlo como un achivo
from sklearn import tree #modelo de arbol
import matplotlib.pyplot #herramienta grafica
import matplotlib.pyplot as plt #herramienta grafica
from sklearn.model_selection import cross_validate #para validacion cruzada 
from sklearn.metrics import accuracy_score #metrica accuracy
from sklearn.metrics import precision_score #metrica precisión
from sklearn.metrics import recall_score #metrica recall
from sklearn.metrics import f1_score #metrica f1 score
import warnings #evita que se llene de warnings el script
warnings.filterwarnings("ignore", category=FutureWarning) #evita que se llene de warnings el script
from sklearn.preprocessing import OrdinalEncoder # preprocesamiento de columna

from sklearn.preprocessing import OneHotEncoder # preprocesamiento de columna



In [3]:
#clono el repositorio

In [4]:
!git clone https://github.com/soyHenry/Datathon.git

fatal: destination path 'Datathon' already exists and is not an empty directory.


In [5]:
#ingesto los datos en dataframe de pandas

In [5]:
df_train=pd.read_csv('Datathon/hospitalizaciones_train.csv')

In [6]:
df_new=df_train.copy() #realizo una copia para trabajar con back up de los datos crudos 

In [8]:
#creo una funcion para adaptar la columna label a categórica

In [7]:
def transf_label(df):
    lista_dias=df['Stay (in days)']
    lista_estancia=[]
    for i in lista_dias:
        if i <=8: 
            i=0
        elif i>8:
            i=1
        lista_estancia.append(i)
    array_estancia=np.array(lista_estancia)
    array_estancia.reshape(1,-1)
    array_estancia.shape
    df['estancia']=array_estancia
    df.drop(columns=['Stay (in days)'], inplace=True)
    return df

In [10]:
#aplico la función

In [8]:
transf_label(df_new)

,Available Extra Rooms in Hospital,Department,Ward_Facility_Code,doctor_name,staff_available,patientid,Age,gender,Type of Admission,Severity of Illness,health_conditions,Visitors with Patient,Insurance,Admission_Deposit,estancia
0,4,gynecology,D,Dr Sophia,0,33070,41-50,Female,Trauma,Extreme,Diabetes,4,Yes,2966.408696,0
1,4,gynecology,B,Dr Sophia,2,34808,31-40,Female,Trauma,Minor,Heart disease,2,No,3554.835677,1
2,2,gynecology,B,Dr Sophia,8,44577,21-30,Female,Trauma,Extreme,Diabetes,2,Yes,5624.733654,0
3,4,gynecology,D,Dr Olivia,7,3695,31-40,Female,Urgent,Moderate,None,4,No,4814.149231,0
4,2,anesthesia,E,Dr Mark,10,108956,71-80,Male,Trauma,Moderate,Diabetes,2,No,5169.269637,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409995,1,gynecology,D,Dr Sarah,1,63105,31-40,Female,Trauma,Minor,Asthama,3,Yes,3902.291076,0
409996,5,gynecology,D,Dr Nathan,5,129292,21-30,Female,Trauma,Extreme,Other,4,No,4771.995223,1
409997,4,gynecology,D,Dr Sarah,9,11399,21-30,Female,Emergency,Minor,Asthama,4,Yes,3816.994210,1
409998,10,gynecology,D,Dr Olivia,10,75003,21-30,Female,Trauma,Moderate,Other,2,Yes,3841.577491,1


In [12]:
#Realizo una transormación de las columnas categoricas a numéricas 
#con label encoder en las no ordinales y ordinal encoder en las que si

In [9]:
le = preprocessing.LabelEncoder()
df_new['Insurance'] = le.fit_transform(df_new['Insurance'])
df_new['Ward_Facility_Code'] = le.fit_transform(df_new['Ward_Facility_Code'])
df_new['gender'] = le.fit_transform(df_new['gender'])

In [10]:
df_new=df_new.drop(columns=['patientid'])

In [11]:
df_new['Department'].unique()

array(['gynecology', 'anesthesia', 'TB & Chest disease', 'radiotherapy',
       'surgery'], dtype=object)

In [12]:
df_new['Type of Admission'].unique()

array(['Trauma', 'Urgent', 'Emergency'], dtype=object)

In [13]:
df_new['doctor_name'].unique()

array(['Dr Sophia', 'Dr Olivia', 'Dr Mark', 'Dr John', 'Dr Sarah',
       'Dr Sam', 'Dr Nathan', 'Dr Simon', 'Dr Isaac'], dtype=object)

In [14]:
df_new['health_conditions'].unique()

array(['Diabetes', 'Heart disease', 'None', 'Other', 'Asthama',
       'High Blood Pressure'], dtype=object)

In [15]:
ohe=OneHotEncoder()

In [16]:
feature_array=ohe.fit_transform(df_new[['Department','doctor_name','Type of Admission','health_conditions']]).toarray()

In [17]:
feature_labels=ohe.categories_

In [18]:
feature_labels=np.array(feature_labels).ravel()

<ipython-input-18-c2a9e482d40a>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  feature_labels=np.array(feature_labels).ravel()


In [19]:
lista_aux=[]
for i in feature_labels:
  for e in i: 
    lista_aux.append(e)
print(lista_aux)

['TB & Chest disease', 'anesthesia', 'gynecology', 'radiotherapy', 'surgery', 'Dr Isaac', 'Dr John', 'Dr Mark', 'Dr Nathan', 'Dr Olivia', 'Dr Sam', 'Dr Sarah', 'Dr Simon', 'Dr Sophia', 'Emergency', 'Trauma', 'Urgent', 'Asthama', 'Diabetes', 'Heart disease', 'High Blood Pressure', 'None', 'Other']


In [20]:
feature_labels=lista_aux

In [21]:
df_extra=pd.DataFrame(feature_array,columns=feature_labels)

In [22]:
enc = OrdinalEncoder()
X2=[['0-10',1],['11-20',2],['21-30',3],['31-40',4],['41-50',5],['51-60',6],['61-70',7],['71-80',8],['81-90',9],['91-100',10]]
enc.fit(X2)
df_new['Age'] = enc.fit_transform(df_new[['Age']])

In [23]:
enc = OrdinalEncoder()
X3=['Minor',1],['Moderate',2],['Extreme',3]
enc.fit(X3)
df_new['Severity of Illness'] = enc.fit_transform(df_new[['Severity of Illness']])

In [24]:
df_new.drop(columns=['Department','doctor_name','Type of Admission','health_conditions'], inplace=True)

In [25]:
df_nuevo=df_new.join(df_extra)

In [26]:
df2=df_nuevo.copy() 

In [27]:
df_nuevo.head(1)

,Available Extra Rooms in Hospital,Ward_Facility_Code,staff_available,Age,gender,Severity of Illness,Visitors with Patient,Insurance,Admission_Deposit,estancia,...,Dr Sophia,Emergency,Trauma,Urgent,Asthama,Diabetes,Heart disease,High Blood Pressure,None,Other
0,4,3,0,4.0,0,0.0,4,1,2966.408696,0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


Las features seleccionadas son: 
Ginecology
Dr. Mark
Age
Anestesia
gender (corr method)
TB & Chest disease (corr method)
Ward_Facility_Code (feature selection method)
Insurance (feature selection method)

In [28]:
df_f=df_nuevo[['anesthesia','gynecology','Ward_Facility_Code','Dr Mark','Age','gender','TB & Chest disease','Insurance','estancia']]

X=df_nuevo[['anesthesia','gynecology','Ward_Facility_Code','Dr Mark','Age','gender','TB & Chest disease','Insurance']]
y=df_nuevo[['estancia']]


In [ ]:
#COMO PUEDE VERSE ESTO SE RELIZA DOS VECES PORQUE SE ESTAN RELIZANDO 2 ARBOLES CON DISTINTA CANTIDAD DE FEATURES CADA UNO 
#PARA COMPARAR SU EFICACIA EN DETECCION DE FALSOS POSITIVOS, ESTE FUE UN PUNTO FLOJO A SOLUCIONAR DURANTE EL PROCESO DE MODELADO

In [29]:
#realizo la primera partición de los datos, luego se hará una validación cruzada

X2_train, X2_test, y2_train, y2_test = train_test_split(X, y, test_size=0.30, random_state=20) 

# datos, dejo un 30% para testear

print(X2_train.shape, y2_train.shape, X2_test.shape, y2_test.shape)

(287000, 8) (287000, 1) (123000, 8) (123000, 1)


In [ ]:
#CON EL SIGUIENTE CODIGO SE DESARROLLO UN ENSAYO DEL HIOPERARÁMETRO OPTIMO CON VALIDACIÓN CRUZADA 

In [30]:
pip install autogluon

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
from autogluon.tabular import TabularPredictor

In [33]:
predictor = TabularPredictor(label='estancia').fit(train_data=df_f)

No path specified. Models will be saved in: "AutogluonModels/ag-20221216_103154/"
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20221216_103154/"
AutoGluon Version:  0.6.1
Python Version:     3.8.16
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Aug 26 08:44:51 UTC 2022
Train Data Rows:    410000
Train Data Columns: 8
Label Column: estancia
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <-->

In [34]:
pred_probs = predictor.predict_proba(df_f)
pred_probs.head(5)

,0,1
0,0.446299,0.553701
1,0.431236,0.568764
2,0.436302,0.563698
3,0.435606,0.564394
4,0.000000,1.000000


In [35]:
results = predictor.fit_summary(show_plot=True)

*** Summary of fit() ***
Estimated performance of each model:
                  model  score_val  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0              CatBoost   0.691463       0.009199    3.219507                0.009199           3.219507            1       True          7
1               XGBoost   0.691463       0.014698    1.320997                0.014698           1.320997            1       True         11
2              LightGBM   0.691463       0.015673    1.611036                0.015673           1.611036            1       True          4
3            LightGBMXT   0.691463       0.018667    3.519128                0.018667           3.519128            1       True          3
4        NeuralNetTorch   0.691463       0.055972  226.819555                0.055972         226.819555            1       True         12
5       NeuralNetFastAI   0.691463       0.056003  246.669050                0.056003         246.

In [36]:
joblib.dump(predictor,'autogluon_categ_trained')

['autogluon_categ_trained']